# Interrogate Notebook
> Make sure you put the .txt file in wordlists folder 


In [2]:
#@title Imports
try : 
  import open_clip
except ImportError:
  !pip install open_clip_torch
  import open_clip

try:
  import torch
except ImportError:
  !pip install torch>=1.9.0
  import torch

try:
  import PIL
  from PIL import Image
except:
  !pip install pillow
  import PIL
  from PIL import Image

import pathlib

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.4 MB 5.1 MB/s 
     |████████████████████████████████| 163 kB 57.3 MB/s 
     |████████████████████████████████| 53 kB 1.4 MB/s 


In [3]:
#@title Load wordlist file function
def load_list(filename):
    with open(filename, 'r', encoding='utf-8', errors='replace') as f:
        items = [line.strip() for line in f.readlines()]
    return items

In [4]:
#@title Interrogate Single Image Function
def interrogate_single_image_topk(model  , 
                             preprocess,
                             image_file = None, 
                             text_file = None ,
                             k = 1 ,
                          ):
  cwd_path = pathlib.Path.cwd().parent
  if image_file is None :

    image_file = cwd_path / 'images' / 'download_blue.png'
  
  if text_file is None :
    text_file = cwd_path / 'wordlists' / 'wordlist.txt'

  image = preprocess(Image.open(image_file)).unsqueeze(0)
  txt_list = load_list(text_file)
  k = min(k, len(txt_list))
  text = open_clip.tokenize(txt_list)
  
  with torch.no_grad():
      image_features = model.encode_image(image)
      text_features = model.encode_text(text)
      image_features /= image_features.norm(dim=-1, keepdim=True)
      text_features /= text_features.norm(dim=-1, keepdim=True)

      similarity = (100.0 * image_features @ text_features.T).softmax(dim=-1)
      values, indices = similarity[0].topk(k)
      # Print the result
      print("\nTop predictions:\n")
      for value, index in zip(values, indices):
          print(f"{txt_list[index]}: {100 * value.item():.2f}%")


In [5]:
#@title List Pretrained model
# Choose one of these models :: model name and pretrained 
open_clip.list_pretrained()

[('RN50', 'openai'),
 ('RN50', 'yfcc15m'),
 ('RN50', 'cc12m'),
 ('RN50-quickgelu', 'openai'),
 ('RN50-quickgelu', 'yfcc15m'),
 ('RN50-quickgelu', 'cc12m'),
 ('RN101', 'openai'),
 ('RN101', 'yfcc15m'),
 ('RN101-quickgelu', 'openai'),
 ('RN101-quickgelu', 'yfcc15m'),
 ('RN50x4', 'openai'),
 ('RN50x16', 'openai'),
 ('RN50x64', 'openai'),
 ('ViT-B-32', 'openai'),
 ('ViT-B-32', 'laion400m_e31'),
 ('ViT-B-32', 'laion400m_e32'),
 ('ViT-B-32', 'laion2b_e16'),
 ('ViT-B-32', 'laion2b_s34b_b79k'),
 ('ViT-B-32-quickgelu', 'openai'),
 ('ViT-B-32-quickgelu', 'laion400m_e31'),
 ('ViT-B-32-quickgelu', 'laion400m_e32'),
 ('ViT-B-16', 'openai'),
 ('ViT-B-16', 'laion400m_e31'),
 ('ViT-B-16', 'laion400m_e32'),
 ('ViT-B-16-plus-240', 'laion400m_e31'),
 ('ViT-B-16-plus-240', 'laion400m_e32'),
 ('ViT-L-14', 'openai'),
 ('ViT-L-14', 'laion400m_e31'),
 ('ViT-L-14', 'laion400m_e32'),
 ('ViT-L-14', 'laion2b_s32b_b82k'),
 ('ViT-L-14-336', 'openai'),
 ('ViT-H-14', 'laion2b_s32b_b79k'),
 ('ViT-g-14', 'laion2b_s12b_

In [6]:
#@title Load model
# Make sure you choosed the desired model
model, _, preprocess = open_clip.create_model_and_transforms('ViT-B-32',pretrained='openai')

100%|████████████████████████████████████████| 354M/354M [00:02<00:00, 164MiB/s]


In [10]:
#@title Run
# run the interrogate_single_image_topk function 
# returns ==> the most matching text in the input text
interrogate_single_image_topk(model = model , preprocess=preprocess , k = 2)


Top predictions:

pixel art: 100.00%
photography: 0.00%
